In [1]:
from pyspark.sql import SparkSession
import pyodbc

In [ ]:
# Configure environment as parameter (pr for production)
env = 'td'

In [ ]:
storage_account = 'sa'+env+'gdwh'
keyvault_name = 'KV-'+env.upper()+'-WEU-GDWH'
accesskey_secret_name = 'secret-accesskey-dls'+env+'gdwh'
keyvault_linkedservice_name = 'ls_KV_WEU_GDWH'

if env == 'td':
   masterdata_sourcefolder = '<source_system>/data/ST2300/MD/'
   transactionaldata_sourcefolder = '<source_system>/data/ST2300/TD/'  
elif env == 'pr':
   masterdata_sourcefolder = '<source_system>/data/S4P100/MD/' 
   transactionaldata_sourcefolder = '<source_system>/data/S4P100/TD/'

accesskey = mssparkutils.credentials.getSecret(keyvault_name,accesskey_secret_name,keyvault_linkedservice_name)
source_folder = 'abfss://enhanced@'+storage_account+'.dfs.core.windows.net/'+masterdata_sourcefolder

# Connec to DB
server = 'synws-'+env+'-weu-gdwh-ondemand.sql.azuresynapse.net'
database = 'Metadata'

if env == 'td':
   username = 'sqladminuser'  
elif env == 'pr':
   username = 'sqladminuser'  

password_secret_name = 'secret-sqladminuser-synws-'+env+'-weu-gdwh'
password = mssparkutils.credentials.getSecret(keyvault_name,password_secret_name,keyvault_linkedservice_name) 
cnxn_string ='Driver={ODBC Driver 17 for SQL Server};Server=tcp:'+server+',1433;Database='+database+';Uid='+username+';Pwd='+password
cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

# List files in source folder
files = mssparkutils.fs.ls(source_folder)

# Extract table name from source folder and create external table and view
for fi in files: 
   table_name = fi.name

   if table_name == 'DD03L_PR' or table_name == 'DD03L' or  table_name == 'DD03M':
      continue

   else:
      # Define queries
      objectexists_query = "SELECT OBJECT_ID('[DSL].[<source_system>_"+table_name+"]','V')"
      exec_statement_vw = "EXEC [dbo].[CREATE_VIEW_SCRIPT_PARQUET_WITHCOLUMNS] @SCHEMA = 'DSL', @TABLE = '"+table_name+ "', @BUSINESSNAMES = 0"

      # Get object_id of view to see if it already exists
      cursor.execute(objectexists_query)
      object_id = cursor.fetchall() 

      # Only create view if it does not yet exists
      if object_id[0][0] is None:
         # Create view
         cursor.execute(exec_statement_vw)
         cursor.commit()
         
      else:
         print("[DSL].[<source_system>_"+table_name+"] already exists.")
